<a href="https://colab.research.google.com/github/isaac-chisaleCS/code-unza25-csc4792-project_team_12-repository/blob/main/code_unza25_csc4792_project_team_12_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.Business Understanding



#1.1 Background
Legal and legislative documents are essential for promoting civic education, transparency, and the rule
of law. However, the volume and complexity of these documents make it difficult for the public,
researchers, and policymakers to easily locate and interpret relevant information.
The DataLab Research Group at The University of Zambia is investigating the application of data mining
techniques to improve the accessibility and organization of such documents.

Two major publicly accessible sources of legal and legislative information in Zambia are:
*  National Assembly of Zambia – Provides legislative bills, acts, and parliamentary debates.

* Zambia Legal Information Institute (ZambiaLII) – Provides court judgments, case summaries, and
other legal references.

The focus of this project is on court judgments in PDF format. These judgments are written in natural
language and cover various categories such as criminal, civil, family, constitutional, and commercial
matters. Classifying these judgments automatically would greatly aid legal practitioners, students,
journalists, and the public in finding and studying relevant cases.
